## Zadanie

Wczytaj dane "Breast Cancer Wisconsin" i przeprowadź preprocesing zgodnie z podanymi poniżej wytycznymi.

Dane znajdują się w repozytorium pod adresem [dane/breast-cancer.data](dane/breast-cancer.data) (tub można je popbrać [tutaj](https://github.com/IS-UMK/wdm_lab_04/blob/master/dane/breast-cancer.data))

Dane zawierają wartości opisujące cechy jąder komórkowych obecnych na obrazie uzyskanym przy badaniu piersi dla dwóch grup pacjentów: `benign` (złośliwy), `malignat` (łagodny).

Oto lista zmiennych:

```
   #  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class                         (2 for benign, 4 for malignant)

```

Wszystkie istotne cechy posiadają wartości numeryczne (całkowite) z zakresu od 1 do 10, ostatnia zmienna zawiera informacje o 2 klasach.

1. Wczytaj zbiór danych ``brast-cancer.data`` do tablicy `DataFrame` używając Pandas. Dane są w formacie zgodnym z CSV (wartości są oddzielone przecinkami). Zwróć uwagę na to, że plik nie posiada nagłówka, tzn. pierwsza linia pliku nie zawiera nazw zmiennych. Nadaj odpowiednie nazwy zmiennych zgodne z opisem danych. 

In [ ]:
import pandas as pd

df = pd.read_csv('./dane/breast-cancer.data', header=None)

df.columns = ['sample-code-number', 'clump-thickness', 'uniformity-of-cell-size',
               'uniformity-of-cell-shape', 'marginal-adhesion', 'single-epithelial-cell-size',
               'bare-nuclei', 'bland-chromatin', 'normal-nucleoli', 'mitosis', 'class']

print(df.head())

   sample-code-number  clump-thickness  uniformity-of-cell-size  \
0             1000025                5                        1   
1             1002945                5                        4   
2             1015425                3                        1   
3             1016277                6                        8   
4             1017023                4                        1   

   uniformity-of-cell-shape  marginal-adhesion  single-epithelial-cell-size  \
0                         1                  1                            2   
1                         4                  5                            7   
2                         1                  1                            2   
3                         8                  1                            3   
4                         1                  3                            2   

  bare-nuclei  bland-chromatin  normal-nucleoli  mitosis  class  
0           1                3                1        1

2. Pierwsza zmienna zawiera liczby porządkowe (``Sample_code_number``) identyfikujące pacjentów. Ta zmienna jest nieistotna dla analizy. Usuń ją ze zbioru.

In [ ]:
df = df.drop(columns=['sample-code-number'], axis=1)

print(df.head())

   clump-thickness  uniformity-of-cell-size  uniformity-of-cell-shape  \
0                5                        1                         1   
1                5                        4                         4   
2                3                        1                         1   
3                6                        8                         8   
4                4                        1                         1   

   marginal-adhesion  single-epithelial-cell-size bare-nuclei  \
0                  1                            2           1   
1                  5                            7          10   
2                  1                            2           2   
3                  1                            3           4   
4                  3                            2           1   

   bland-chromatin  normal-nucleoli  mitosis  class  
0                3                1        1      2  
1                3                2        1      2  
2       

3. Wartości brakujące w tekstowym pliku wejściowym kodowane są za pomocą znaku zapytania (`?`). Wyświtl liczbę przypadków z wartościami brakującymi oraz liczbę wartości brakujących w każdej ze zmiennych.

In [ ]:
count = (df == '?').sum().sum()
print(f'Liczba brakujących wartości = {count}')

counts_per_column = (df == '?').sum(axis=0);
print(counts_per_column)

Liczba brakujących wartości = 16
clump-thickness                 0
uniformity-of-cell-size         0
uniformity-of-cell-shape        0
marginal-adhesion               0
single-epithelial-cell-size     0
bare-nuclei                    16
bland-chromatin                 0
normal-nucleoli                 0
mitosis                         0
class                           0
dtype: int64


4. Pozbądź się wartości brakujących ze zbioru danych. Usuń przypadki z brakami lub zamień wartości brakujące na wartość najczęściej występującą, czyli na wartość mody (dominanty), którą można wyznaczyć funkcją [DataFrame.mode()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mode.html)).  
Po transformacji wszystkie zmienne powinny być zmiennymi numerycznymi. Jeśli tak nie jest, to przetransformuj je do postaci numerycznej. 

In [ ]:
import numpy as np

all_values_excluding_question = [value for value in df.values.flatten() if value != '?']

mode = pd.Series(all_values_excluding_question).mode()[0]
print(f'Moda = {mode}')
df = df.replace('?', mode)
count = (df == '?').sum().sum()
print(f'Liczba brakujących wartości = {count}')

Moda = 1
Liczba brakujących wartości = 0


In [93]:
non_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()
print(non_numeric)

['bare-nuclei']


In [ ]:
print(df[non_numeric[0]].dtype)

df[non_numeric[0]] = pd.to_numeric(df[non_numeric[0]], errors='coerce')

print(df[non_numeric[0]].dtype)

print(df.head())

object
int64
   clump-thickness  uniformity-of-cell-size  uniformity-of-cell-shape  \
0                5                        1                         1   
1                5                        4                         4   
2                3                        1                         1   
3                6                        8                         8   
4                4                        1                         1   

   marginal-adhesion  single-epithelial-cell-size  bare-nuclei  \
0                  1                            2            1   
1                  5                            7           10   
2                  1                            2            2   
3                  1                            3            4   
4                  3                            2            1   

   bland-chromatin  normal-nucleoli  mitosis  class  
0                3                1        1      2  
1                3                2        

5. Sprawdź, czy zbiór danych zawiera przypadki odstające lub nietypowe. Spodziewamy się, że wszystkie zmienne (oprócz klasy) mają wartości całkowite w zakresie od 1 do 10. Usuń ze zbioru danych wszystkie przypadki, które uznasz za niespójne ze spodziewanym rozkładem danych. 

In [ ]:
columns_to_check = [col for col in df.columns if col != 'class']

for col in columns_to_check:
    outside_range = (df[col] < 1) | (df[col] > 10)
    df.loc[outside_range, col] = np.nan

    replaced_count = outside_range.sum()
    print(f'Zmienna {col} - liczba zastąpionych wartości = {replaced_count}')

Zmienna clump-thickness - liczba zastąpionych wartości = 0
Zmienna uniformity-of-cell-size - liczba zastąpionych wartości = 0
Zmienna uniformity-of-cell-shape - liczba zastąpionych wartości = 1
Zmienna marginal-adhesion - liczba zastąpionych wartości = 0
Zmienna single-epithelial-cell-size - liczba zastąpionych wartości = 0
Zmienna bare-nuclei - liczba zastąpionych wartości = 0
Zmienna bland-chromatin - liczba zastąpionych wartości = 0
Zmienna normal-nucleoli - liczba zastąpionych wartości = 0
Zmienna mitosis - liczba zastąpionych wartości = 0


6. Usuń ze zbioru przypadki odstające, których wartości zmiennych wykraczają poza przedział $(\bar{x}-3\cdot\sigma, \bar{x}+ 3\cdot\sigma)$, gdzie $\bar{x}$ to wartość średnia zmiennej, $\sigma$ to odchylnie standardowe zmiennej.

In [ ]:
rows_to_keep = pd.Series(True, index=df.index)

for col in columns_to_check:
    mean = df[col].mean()
    std = df[col].std()

    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std

    mask = (df[col] >= lower_bound) & (df[col] <= upper_bound)
    rows_to_keep &= mask

    cases_found = (~mask).sum()
    print(f'Zmienna {col} - liczba przypadków = {cases_found}')

print(f'Przed usunięciem przypadków = {len(df)}')
df = df[rows_to_keep]
print(f'Po usunięciu przypadków = {len(df)}')

Zmienna clump-thickness - liczba przypadków = 0
Zmienna uniformity-of-cell-size - liczba przypadków = 0
Zmienna uniformity-of-cell-shape - liczba przypadków = 1
Zmienna marginal-adhesion - liczba przypadków = 0
Zmienna single-epithelial-cell-size - liczba przypadków = 31
Zmienna bare-nuclei - liczba przypadków = 0
Zmienna bland-chromatin - liczba przypadków = 0
Zmienna normal-nucleoli - liczba przypadków = 0
Zmienna mitosis - liczba przypadków = 31
Przed usunięciem przypadków = 699
Po usunięciu przypadków = 647


In [97]:
print(df.head())

   clump-thickness  uniformity-of-cell-size  uniformity-of-cell-shape  \
0              5.0                      1.0                       1.0   
1              5.0                      4.0                       4.0   
2              3.0                      1.0                       1.0   
3              6.0                      8.0                       8.0   
4              4.0                      1.0                       1.0   

   marginal-adhesion  single-epithelial-cell-size  bare-nuclei  \
0                1.0                          2.0          1.0   
1                5.0                          7.0         10.0   
2                1.0                          2.0          2.0   
3                1.0                          3.0          4.0   
4                3.0                          2.0          1.0   

   bland-chromatin  normal-nucleoli  mitosis  class  
0              3.0              1.0      1.0      2  
1              3.0              2.0      1.0      2  
2 

7. Usuń ze zbioru danych duplikaty, tzn. powtarzające sie przypadki. 

In [98]:
print(f'Przed usunięciem duplikatów = {len(df)}')
df = df.drop_duplicates()
print(f'Po usunięciu duplikatów = {len(df)}')

Przed usunięciem duplikatów = 647
Po usunięciu duplikatów = 406


8. Uzyskany zbiór danych wyeksportuj do pliku tekstowego w formacie CSV o nazwie ``breast-cancer.csv``. Plik wynikowy powinien zawierać nagłowek z nazwami zmiennych, średnik (`;`) jako separator oraz nie powinien zawierać zmiennej indeksującej.  Zabacz funkcję [DataFrame.to_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html). Wynikowy plik danych umieść w repozytorium GitHub.




In [ ]:
df.to_csv('breast-cancer.csv', sep=';', index=False, header=True)